In [1]:
import os
import sys 
os.chdir("/workspaces/dev/modules")
sys.path.append("/workspaces/dev/models/R-CNN")
os.environ["TF_XLA_FLAGS"] = "--tf_xla_auto_jit=2 --tf_xla_cpu_global_jit=false"
os.environ["XLA_FLAGS"] = "--xla_cpu_multi_thread_eigen=false intra_op_parallelism_threads=1 inter_op_parallelism_threads=1"

In [2]:
from tensorflow import keras
import tensorflow as tf
from datetime import datetime

2025-03-16 19:15:46.765486: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-16 19:15:46.775157: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742152546.787708   49905 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742152546.791211   49905 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-16 19:15:46.803464: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
from Pills.GenerativeSequence import GenerativeSequence
from PillNetRoI import PillNetRoI

In [4]:
SOURCE_PATH = '/workspaces/dev/datasets/pills/data'
LABEL_PATH = '/workspaces/dev/datasets/pills/class_label.csv'
BACKGROUND_IMAGEES_PATH = '/workspaces/dev/data'

In [5]:
TRAIN_DATA_SIZE = 5000
VALIDATION_DATA_SIZE = 100
BATCH_SIZE = 32
INPUT_SIZE = (256, 256, 3)

In [ ]:
model = PillNetRoI()
model.compile(jit_compile=False)
# model.compile(jit_compile=False, run_eagerly=True)
model.build(BATCH_SIZE)
model.summary()

I0000 00:00:1742152549.002402   49905 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13512 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4080, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "pill_net_ro_i"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 16)   │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 128, 128, 16)   │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 128, 128, 16)   │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 128, 128, 16)   │         9,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 128, 128, 16)   │         2,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_2 (Sequential)       │ (None, 128, 128, 16)   │         6,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_3 (Sequential)       │ (None, 128, 128, 16)   │         8,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_4 (Sequential)       │ (None, 128, 128, 1)    │         9,257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 110,981 (433.52 KB)

 Trainable params: 36,993 (144.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 73,988 (289.02 KB)

In [7]:
train_sequence = GenerativeSequence(
  TRAIN_DATA_SIZE, BATCH_SIZE, input_size = INPUT_SIZE[:2],material_path=SOURCE_PATH,
  label_path=LABEL_PATH, background_images_path=BACKGROUND_IMAGEES_PATH,
  use_multiprocessing=True, workers = 12
)
validation_sequence = GenerativeSequence(
  VALIDATION_DATA_SIZE, BATCH_SIZE, input_size = INPUT_SIZE[:2],material_path=SOURCE_PATH,
  label_path=LABEL_PATH, background_images_path=BACKGROUND_IMAGEES_PATH,
  use_multiprocessing=True, workers = 8
)
train_dataset = tf.data.Dataset.from_generator(
  lambda: train_sequence, 
  output_signature=(
    tf.TensorSpec(shape=(BATCH_SIZE, INPUT_SIZE[0], INPUT_SIZE[1], INPUT_SIZE[2]), dtype=tf.float32),
    {
      "roi": tf.TensorSpec(shape=(BATCH_SIZE, INPUT_SIZE[0], INPUT_SIZE[1]), dtype=tf.bool),
      "centroid": tf.TensorSpec(shape=(BATCH_SIZE * 16, 2), dtype=tf.float32),
      "detection": tf.TensorSpec(shape=(BATCH_SIZE * 16, 4), dtype=tf.float32),
      "segmentation": tf.TensorSpec(shape=(BATCH_SIZE * 16, INPUT_SIZE[0]//4, INPUT_SIZE[1]//4), dtype=tf.bool),
      "classification": tf.TensorSpec(shape=(BATCH_SIZE * 16), dtype=tf.uint32)
    }
  )
)

In [ ]:
checkpoint_cb = keras.callbacks.ModelCheckpoint(
  filepath=f"/workspaces/dev/models/R-CNN/checkpoints/PillNetRoI_{datetime.now().strftime('%Y_%m_%d')}.keras",
  monitor="roi_Dice",
  mode = 'max',
  save_best_only=True,
  save_weights_only=False,
  verbose = 1
)

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(
  monitor="roi_Dice",
  mode = 'max',
  patience=10,
  restore_best_weights=True,
  verbose = 1
)

In [10]:
model.fit(
  train_sequence,
  validation_data=validation_sequence,
  epochs=1,
  callbacks=[checkpoint_cb, early_stopping_cb],
  verbose=1
)

I0000 00:00:1742152595.239186   49905 cuda_dnn.cc:529] Loaded cuDNN version 90300


157/157 ━━━━━━━━━━━━━━━━━━━━ 124s 727ms/step - roi_Binary: 0.6894 - roi_Dice: 0.4551 - roi_IoU: 0.3489 - roi_Pixel: 1.1597e-04 - val_roi_Binary: 0.8023 - val_roi_Dice: 0.6844 - val_roi_IoU: 0.5752 - val_roi_Pixel: 0.0054


/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/model_checkpoint.py:209: UserWarning: Can save best model only with roi available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `roi` which is not available. Available metrics are: roi_Binary,roi_Dice,roi_IoU,roi_Pixel,val_roi_Binary,val_roi_Dice,val_roi_IoU,val_roi_Pixel
  current = self.get_monitor_value(logs)


In [11]:
raise Exception("End")

Exception: End

In [13]:
model.save(f"/workspaces/dev/models/R-CNN/checkpoints/PillNetRoI_sub_{datetime.now().strftime('%Y_%m_%d_sub')}.keras")